1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다
   - 토큰 수 초과로 답변을 생성하지 못할 수 있음
   - 문서가 길면 (input이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달


# 1. 문서의 내용을 읽는다.


In [ ]:
# python-docx 라이브러리를 사용하여 .docx 파일 읽기
from docx import Document

# tax.docx 파일 로드
document = Document("./tax.docx")

# 전체 텍스트를 저장할 변수 초기화
full_text = ""

# 문서의 모든 문단(paragraph)을 순회하면서 텍스트 추출
for idx in range(len(document.paragraphs)):
    print(document.paragraphs[idx].text)
    full_text += f"{document.paragraphs[idx].text}\n"

# 2. 문서를 쪼갠다


In [ ]:
import tiktoken


def split_text(full_text, chunk_size):
    """
    텍스트를 지정된 토큰 크기로 분할하는 함수

    Args:
        full_text: 분할할 전체 텍스트
        chunk_size: 각 청크의 최대 토큰 수

    Returns:
        text_list: 분할된 텍스트 조각들의 리스트
    """
    # gpt-4o-mini 모델의 토크나이저 가져오기
    encoder = tiktoken.encoding_for_model("gpt-4o-mini")

    # 전체 텍스트를 토큰으로 인코딩
    total_encoding = encoder.encode(full_text)
    total_token_count = len(total_encoding)

    # 분할된 텍스트를 저장할 리스트 초기화
    text_list = []

    # chunk_size 단위로 토큰을 분할
    for i in range(0, total_token_count, chunk_size):
        # 현재 청크의 토큰 추출
        chunk = total_encoding[i : i + chunk_size]
        # 토큰을 다시 텍스트로 디코딩
        decoded = encoder.decode(chunk)
        text_list.append(decoded)

    return text_list

In [ ]:
# 전체 텍스트를 1500 토큰 크기의 청크로 분할
chunk_list = split_text(full_text, chunk_size=1500)

# 분할된 청크 리스트 확인
chunk_list

# 3. 임베딩


In [ ]:
import chromadb

chroma_client = chromadb.Client()

In [ ]:
collection_name = "tax_collection"

tax_collection = chroma_client.create_collection(collection_name)

In [ ]:
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()  # .env 파일에서 환경 변수 로드

openai_embedding = OpenAIEmbeddingFunction(
    model_name="text-embedding-3-large",
)


In [ ]:
tax_collection = chroma_client.get_or_create_collection(collection_name)

In [ ]:
id_list = []

for index in range(len(chunk_list)):
    id_list.append(f"{index}")

In [ ]:
len(id_list)

In [ ]:
len(chunk_list)

In [ ]:
tax_collection.add(
    documents=chunk_list,
    ids=id_list,
)

# 4. 유사도 검색


In [20]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_doc = tax_collection.query(
    query_texts=query,
)

# 5. LLM 질의


In [21]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": f"당신은 한국의 소득세 전문가 입니다. 아래 내용을 참고해서 질문에 답변해 주세요. {retrieved_doc['documents'][0]}",
        },
        {"role": "system", "content": query},
    ],
)

In [22]:
response.choices[0].message.content

'소득세는 과세표준과 세율에 따라 계산됩니다. 2023년 기준으로 한국의 소득세는 누진세율 구조를 가지고 있습니다. 일반적인 소득세율은 다음과 같습니다:\n\n1. 1,200만원 이하: 6%\n2. 1,200만원 초과 ~ 4,600만원 이하: 15%\n3. 4,600만원 초과 ~ 8,800만원 이하: 24%\n4. 8,800만원 초과 ~ 1억5천만원 이하: 35%\n5. 1억5천만원 초과: 38%\n\n**연봉이 5천만원인 직장인의 소득세를 계산해 보겠습니다**:\n\n1. **소득금액**: 50,000,000원\n\n2. **과세표준**:\n   - 연간 소득금액: 50,000,000원\n   - 기본공제 및 인적공제(예: 기본공제 150만원) 등을 고려하여 과세표준을 결정합니다.\n   - 일반적으로 기본 공제 후 과세표준은 약 48,500,000원이 됩니다. (정확한 공제액은 개인의 상황에 따라 다름)\n\n3. **세액 계산**:\n   - 1,200만원까지: 1,200만원 × 6% = 72,000원\n   - 1,200만원 초과 4,600만원까지 (3,400만원): 3,400만원 × 15% = 510,000원\n   - 4,600만원 초과 (1,400만원): 1,400만원 × 24% = 336,000원\n\n4. **총 세액**:\n   - 72,000원 + 510,000원 + 336,000원 = 918,000원\n\n5. **최종 세액**: \n   - 계산된 총 세액은 **약 918,000원**입니다.\n   - 여기서 추가적으로 주민세(소득세의 10% 정도)가 추가로 발생할 수 있습니다.\n\n따라서, 대략적인 연봉 5천만원 직장인의 소득세는 **약 918,000원**이 될 것으로 예상됩니다. (정확한 금액은 개인의 공제 상황에 따라 달라질 수 있습니다). 추가적으로 공제가 더 있다면 소득세는 더 낮아질 수 있습니다.'